In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import regex as re
import requests
import pandas as pd

Fetch HTML of Main Website

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://mrcc.purdue.edu/research/awssi/indexAwssi.jsp")
driver.find_element("id", "showStations").click()
soup = BeautifulSoup(driver.page_source)


Fetch Links of All End-of-Year Station Data

In [ ]:
re.findall(r'rawssi([^"]*)',str(soup.find_all(id="stations")[0]))
csv_links = []
for i in re.findall(r'rawssi([^"]*)',str(soup.find_all(id="stations")[0])):
    z = "https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi" + str(i)
    csv_links.append(z)

csv_links


Creating Main Dataframe

In [ ]:
columns = ["Season", "Start", "End", "Length", "Msng", "TMax", "TMin", "TempSc", "Sfall", "Sdepth", "SnowSc", "AWSSI", "Events", "RAWSSI", "Location"]
data = pd.DataFrame(columns=columns)
for x in csv_links:
    df = pd.read_csv(x, skiprows=1)
    df2 = pd.read_csv(x)
    df['Location']=re.findall(r'"([^"]*)' ,df2.columns[2])[0]
    df.rename(columns={"   Season":"Season", "     Start":"Start", "  Msng": "Msng", "  Tmax": "Tmax", "  Tmin":"Tmin", " Sfall":"Sfall", "       End":"End", " AWSSI":"AWSSI"}, inplace=True)
    data= pd.concat([data,df])

In [ ]:
data.to_csv("Winter-Data.csv")